In [ ]:
%load_ext autoreload
%autoreload 2
# %load_ext line_profiler

import math
from pathlib import Path

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import toolbox_sxp as sxp

%matplotlib widget

In [ ]:
# dataPath = '../../' # Put in Path to a storage of at least 20 Gbyte free space.
# if not os.path.exists(dataPath + "/WSe2.zip"):
#     os.system(f"curl --output {dataPath}/WSe2.zip https://zenodo.org/record/6369728/files/WSe2.zip")
# if not os.path.isdir(dataPath + "/Scan049_1") or not os.path.isdir(dataPath + "energycal_2019_01_08/"):
#     shutil.unpack_archive(dataPath + '/WSe2.zip', extract_dir=dataPath)

In [ ]:
data = Path("/home/jimenezd/_scratch/SXP/Datasets")
# The Scan directory
files_dir = data / "Scan049_1"

# Load data

In [ ]:
polars_df = sxp.get_df(files_dir, apply_jitter=True, concat_raw=True)

In [ ]:
polars_df.head(4)

In [ ]:
file_num = 2

files_sorted = sxp.reader._sort_dir(files_dir)
offset = sxp.reader._get_chunks(files_sorted)
_start = sum(offset[v] for v in range(file_num))
_end = sum(offset[v] for v in range(file_num + 1))
dfpid_2 = polars_df[_start:_end, :]

In [ ]:
# from config file
_axes = ("X", "Y", "t", "ADC")
_bins = (80, 80, 80, 80)
_ranges = ((0, 1800), (0, 1800), (65000, 69000), (0, 9000))


fig, axs = plt.subplots(2, 2, figsize=(10, 6))
fig.suptitle(f"Event histograms of file {file_num}")
axs = axs.flatten()

for i, k in enumerate(_axes):
    axs[i].hist(
        dfpid_2.select(f"{k}_jitter"),
        bins=_bins[i],
        range=_ranges[i],
        label=k,
        histtype="bar",
        edgecolor="#6A9662",
        color="#DDFFDD",
    )
    axs[i].legend(frameon=False)

# Distortion correction and Momentum Calibration workflow

In [ ]:
_axes = ("X", "Y", "t")
_bins = (512, 512, 300)
_ranges = ((-256, 1792), (-256, 1792), (66000, 68000))

### Step 1

In [ ]:
pbinning_Jit = sxp.binning.pbinning(
    files_sorted,
    nproc=60,
    axes=_axes,
    bins=_bins,
    ranges=_ranges,
    jittered=True,
)

In [ ]:
plane = 33
width = 10
V = pbinning_Jit["binned"]
E_slice = V[:, :, plane : plane + width].sum(axis=(2)).T

In [ ]:
f, ax = plt.subplots(figsize=(4, 4))
ax.imshow(E_slice, origin="lower", cmap="terrain_r")
ax.set_title(f"Plane[{plane}:{plane+width}]");

### Step 2

In [ ]:
features = sxp.utils.annotate_features(E_slice, threshold=15, order="ccw")

targets = sxp.utils.rotational_array_generation(
    features, symmetry_order=6, export_centre=True
)

mod_img, ext = sxp.transformations.translate_and_rotate(
    E_slice,
    shift=features["centre"][:-1],
    angle=0,
    rotation_center=features["centre"][:-1],
    scaling=1.0,
)

warped_img, deformation_field = sxp.transformations.generate_splinewarp(
    mod_img, features, targets, get_dfield=True
)

In [ ]:
f, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(12, 4), constrained_layout=True)

im0 = ax0.imshow(E_slice, origin="lower", cmap="terrain_r")
ax0.set_title("Unmodified image\nat its original location")

# fixed lines
ax0.axhline(y=E_slice.shape[1] / 2, color="r", alpha=0.3)
ax0.axvline(x=E_slice.shape[0] / 2, color="r", alpha=0.3)
center = (E_slice.shape[0] / 2, E_slice.T.shape[1] / 2)
ax0.axline(xy1=center, slope=math.tan(math.radians(60)), alpha=0.3)
ax0.axline(xy1=center, slope=math.tan(math.radians(120)), alpha=0.3)

hexagon0 = patches.RegularPolygon(
    (255, 255),
    numVertices=6,
    radius=100,
    orientation=math.radians(30),
    alpha=0.2,
    edgecolor="k",
)
ax0.add_patch(hexagon0)


im1 = ax1.imshow(mod_img, origin="lower", cmap="terrain_r", extent=ext)
ax1.set_title("Unmodified image\ntranslated to (0, 0)")

# fixed lines
ax1.axhline(y=0, color="r", alpha=0.3)
ax1.axvline(x=0, color="r", alpha=0.3)
ax1.axline((0, 0), slope=math.tan(math.radians(60)), alpha=0.3)
ax1.axline((0, 0), slope=math.tan(math.radians(120)), alpha=0.3)

hexagon1 = patches.RegularPolygon(
    (0, 0),
    numVertices=6,
    radius=100,
    orientation=math.radians(30),
    alpha=0.2,
    edgecolor="k",
)
ax1.add_patch(hexagon1)


ax2.imshow(warped_img, origin="lower", extent=ext, cmap="terrain_r")
ax2.set_title("Warped image\ntranslated to (0,0)")

# fixed lines
ax2.axhline(0, color="r", alpha=0.3)
ax2.axvline(0, color="r", alpha=0.3)
ax2.axline((0, 0), slope=math.tan(math.radians(60)), alpha=0.3)
ax2.axline((0, 0), slope=math.tan(math.radians(120)), alpha=0.3)

hexagon2 = patches.RegularPolygon(
    (0, 0),
    numVertices=6,
    radius=90,
    orientation=math.radians(30),
    alpha=0.2,
    edgecolor="k",
)
ax2.add_patch(hexagon2);

### Step 3

In [ ]:
# Original grid (unmodified)
imw, imh = E_slice.shape
x, y = np.meshgrid(
    np.arange(imw, dtype=np.float32), np.arange(imh, dtype=np.float32), copy=False
)

# Deformation field
x_def, y_def = deformation_field

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(12, 4))

ax0.scatter(x[::10, ::10], y[::10, ::10], s=1)
ax0.set_title("Original grid of the image")
ax0.axhline(y=250, color="r", alpha=0.3)
ax0.axvline(x=250, color="r", alpha=0.3)

ax1.scatter(x_def[::10, ::10], y_def[::10, ::10], s=1)
ax1.set_title("Deformation field after warping")
ax1.axhline(y=250, color="r", alpha=0.3)
ax1.axvline(x=250, color="r", alpha=0.3);

### Step 4

In [ ]:
inv_dfield = sxp.utils.generate_inverse_dfield(
    r_dfield=y_def,
    c_dfield=x_def,
    bin_ranges=((-256, 1792), (-256, 1792)),
    detector_ranges=((0, 2048), (0, 2048)),
)

In [ ]:
applied_dfield = sxp.utils.apply_dfield(
    df=polars_df,
    dfield=inv_dfield,
    x_col="X_jitter",
    y_col="Y_jitter",
    detector_ranges=((0, 2048), (0, 2048)),
)

In [ ]:
k_space = sxp.calibration.momentum_cal(E_slice, point = features['vertices'][2, :2], k_distance=3.28, df=applied_dfield)

# Energy calibration

In [ ]:
data = Path("/home/jimenezd/_scratch/SXP/Datasets")
# The Energycal directory
files_dir = data / "energycal_2019_01_08"

_axes = ("t",)
_bins = (1000,)
_ranges = ((64000, 75000),)
voltages = np.arange(-12, -23, -1, dtype=np.int8)

In [ ]:
files_sorted = sxp.reader._sort_dir(files_dir)

my_parp = sxp.binning.pbinning(
    files_sorted,
    nproc=len(files_sorted),
    axes=_axes,
    bins=_bins,
    ranges=_ranges,
    jittered=True,
    sum_binned=False,
    normalize=True,
    smooth=True,
)

In [ ]:
f, ax = plt.subplots(figsize=(9, 4), constrained_layout=True)
cmap = plt.get_cmap("jet_r")
_colors = (
    cmap(float(idx) / len(my_parp["binned"])) for idx, _ in enumerate(my_parp["binned"])
)
ax.set_prop_cycle(color=_colors)

ax.plot(
    my_parp["axes"]["t"],
    my_parp["binned"].T,
    alpha=0.7,
    ls="--",
    linewidth=0.7,
    label=[f"{v} V" for v in voltages],
)

ax.set_xlabel("Time of flight", fontsize=15)
ax.set_ylabel("Normalized intensity", fontsize=15)
ax.legend(frameon=False)
ax.grid();

In [ ]:
my_pks = sxp.calibration.peaks_search(
    my_parp["axes"]["t"],
    my_parp["binned"],
    initial_guess_range=(65000, 65250),
    plot=True,
)

In [ ]:
E_calibration = sxp.calibration.energy_cal(
    my_parp["axes"]["t"], my_parp["binned"], my_pks, voltages, plot=True
)